# imports

In [ ]:
from pandas import read_excel
import numpy as np
from numpy.linalg import eigh
import scipy as sp
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.pyplot.ion()
%matplotlib inline
import pickle

from qlp.mds import graph_tools as gt
from qlp.mds.qubo import get_mds_qubo
from qlp.mds.mds_qlpdb import QUBO_to_Ising, find_offset, AnnealOffset

from qlp.tdse import AnnealSchedule, TDSE, embed_qubo_example

import networkx as nx
from networkx import Graph
from networkx.drawing.nx_pylab import draw

# Setup

## Graph specific parameters 

In [ ]:
nvertices = 2
graph, tag = gt.generate_nn_graph(nvertices)
directed = False

## ILP / QUBO setup

In [ ]:
penalty = 2
qubo = get_mds_qubo(
    graph, directed=directed, penalty=penalty, triangularize=True, dtype="d"
)
qubo_constant = penalty * nvertices

## Set up ising model

In [ ]:
Jij, hi, c = QUBO_to_Ising(qubo.todense().tolist())

embed = True  # nvertices = [2] available
if embed:
    qubo, embedding = embed_qubo_example(nvertices)
    hi_for_offset = np.array(
        [
            hi[key]
            for key in np.sort(list(embedding.keys()))
            for count in range(len(embedding[key]))
        ]
    )
    Jij, hi, c = QUBO_to_Ising(qubo.todense().tolist())
else:
    
    hi_for_offset = hi
    
n = len(qubo.todense())

## Anneal specific parameters

In [ ]:
annealing_time = 0.0003  # microseconds set by normalized_time [0, 1]
normalized_time = [-0.2, 1.2]  # make larger range for complete offsets

# assume annealing_time unit in microsecond and energy unit in Ghz
def energyscaleconversion(annealing_time):
    return annealing_time * 1000.0 * (2.0 * np.pi)

energyscale = energyscaleconversion(annealing_time)
ising_params = {"Jij": Jij, "hi": hi, "c": c, "energyscale": energyscale}

## Anneal curve specific parameters

In [ ]:
offset = "negbinary"
anneal_curve = "dwave"  # "linear", "logistic" or "dwave".
fill_value = "truncate"  # "extrapolate" or "truncate" the offset. Extrapolate to simulate DWave offset.

## Parameters for numerical solve of ODE

In [ ]:
solver_params = dict()
solver_params["method"] = "RK45"
solver_params["rtol"] = 1e-6
solver_params["atol"] = 1e-7

## Wave function parameters

In [ ]:
pure_tdse = True
mixed_tdse = False
temp = 15e-3  # temperature in kelvins for mixed tdse
initial_wavefunction = "true"  # "true" or "transverse"

## Output parameters

In [ ]:
degeneracy_tol = 2e-2  # label energy as degenerate to g.s. if within this fraction

debug = True  # prints debug statements to screen
save_result = True

# Run computations

In [ ]:
# brute force solving time-dependent schrodinger equation to simulate quantum annealing for small systems
result = dict()
offset_min_list = -0.05 * np.arange(2)

for offset_min in offset_min_list:
    
    print(offset_min)
    offset_min = offset_min
    offset_range = -2 * offset_min
   
    offset_params = {
        "normalized_time": normalized_time,
        "offset": offset,
        "hi_for_offset": hi_for_offset,
        "offset_min": offset_min,
        "offset_range": offset_range,
        "fill_value": fill_value,
        "anneal_curve": anneal_curve,
    }

    # instantiate ode solver
    tdse = TDSE(n, ising_params, offset_params, solver_params)

    # plot anneal schedule
    tdse.AS.plot(normalized_time)

    ############################################################
    ###### code block for thermal mixed state simultation ######
    ############################################################
    if mixed_tdse:
        # initial density matrix
        rho = tdse.init_densitymatrix(temp, initial_wavefunction, debug=debug)

        # solve
        sol_densitymatrix = tdse.solve_mixed(rho)

        # overlap with Ising ground state
        gs_idx, Isingenergy, Isingeigv = tdse.ground_state_degeneracy(
            tdse.IsingH, degeneracy_tol, debug=debug
        )
        project = sum(
            [np.kron(Isingeigv[:, idx], np.conj(Isingeigv[:, idx])) for idx in gs_idx]
        )
        prob = np.asarray(
            [
                np.absolute((np.dot(np.conj(project), sol_densitymatrix.y[:, i])))
                for i in range(sol_densitymatrix.t.size)
            ]
        )
        plt.plot(sol_densitymatrix.t, prob)

    ############################################################
    ###### end of code block for thermal mixed state simultation ######
    ############################################################

    if pure_tdse:
        # Initial wavefunction
        y1 = tdse.init_wavefunction(initial_wavefunction)

        # solve
        sol = tdse.solve_pure(y1, ngrid=2, debug=debug)

        # brute force diagonalization Ising hamiltonain
        ground_state_idx, energy, eigv = tdse.ground_state_degeneracy(
            tdse.IsingH, degeneracy_tol, debug=debug
        )

        # plot probability overlap with Ising ground state
        overlap = np.zeros(sol.t.size)
        overlap_annealH = np.zeros(sol.t.size)
        degen_number = np.zeros(sol.t.size)
        for i, t in enumerate(sol.t):
            # overlap with target Hamiltonian ground state
            overlap[i] = tdse.calculate_overlap(eigv, sol.y[:, i], ground_state_idx)
            # overlap with time-dependent Hamiltonian ground state
            annealH_gs_idx, eigval_annealH, eigv_annealH = tdse.ground_state_degeneracy(
                tdse.annealingH(t), degeneracy_tol
            )
            degen_number[i] = len(annealH_gs_idx)
            overlap_annealH[i] = tdse.calculate_overlap(
                eigv_annealH, sol.y[:, i], annealH_gs_idx
            )

        if save_result:
            with open(
                f"../qlp/mds/tdse/{embed}_{initial_wavefunction}_{offset}_{offset_min}.pickle",
                "wb",
            ) as file:
                rpickle = dict()
                rpickle["tdse"] = tdse
                rpickle["sol"] = sol
                rpickle["overlap"] = overlap
                rpickle["overlap_annealH"] = overlap_annealH
                rpickle["degen_number"] = degen_number
                pickle.dump(rpickle, file)

In [ ]:
result = dict()
proba = dict()
proba_annealH = dict()
degen_number = dict()
plt.figure("prob(s)")
ax = plt.axes()
for offset_min in offset_min_list:
    with open(
        f"../qlp/mds/tdse/{embed}_{initial_wavefunction}_{offset}_{offset_min}.pickle",
        "rb",
    ) as file:
        rpickle = pickle.load(file)
        offset_range = -2 * offset_min
        result[offset_range] = rpickle["tdse"]
        proba[offset_range] = rpickle["overlap"][-1]
        proba_annealH[offset_range] = rpickle["overlap_annealH"][-1]
        degen_number[offset_range] = rpickle["degen_number"]
    ax.errorbar(x=rpickle["sol"].t, y=rpickle["overlap"], label=offset_range)
ax.legend()
ax.set_xlabel("time s")
ax.set_ylabel(r"$P= |<\psi (s) | gnd >|^2$")
plt.title("Probability of reaching Ising ground state")
plt.savefig("./proba.pdf")
plt.show()

plt.figure("annealH prob(s)")
ax = plt.axes()
for offset_min in offset_min_list:
    ax.errorbar(x=rpickle["sol"].t, y=rpickle["overlap_annealH"], label=offset_range)
ax.legend()
ax.set_xlabel("time s")
ax.set_ylabel(r"$P= |<\psi (s) | \psi_0(s) >|^2$")
plt.title("Probability of wavefunction in H(s) ground-state")
plt.show()

plt.figure("degeneracy number")
ax = plt.axes()
for offset_min in offset_min_list:
    ax.errorbar(x=rpickle["sol"].t, y=rpickle["degen_number"], label=offset_range)
ax.legend()
ax.set_xlabel("time s")
ax.set_ylabel(r"ground-state degeneracy")
plt.show()

plt.figure("prob final")
ax = plt.axes()
ax.errorbar(x=list(proba.keys()), y=list(proba.values()), marker="o")
ax.set_xlabel("offset range")
ax.set_ylabel("s=1 probability")
plt.title(offset)
plt.show()

# plot last anneal schedule
plt.figure()
ax = plt.axes()
X = np.linspace(*normalized_time)
yA = np.array([tdse.AS.A(Xi) for Xi in X])
yB = np.array([tdse.AS.B(Xi) for Xi in X])
for qubit in range(len(yA[0])):
    ax.errorbar(x=X, y=yA[:, qubit])
    ax.errorbar(x=X, y=yB[:, qubit], ls="--")

In [ ]:
# unit conversion
# sol.y = [GHz / h]
make_dimensionless = 1 / energyscale * 1 / tdse.AS.B(normalized_time[1])[0]

fig = plt.figure()
ax = plt.axes()
num_es = 10
gap = dict()
for key in result.keys():
    x = []
    y = {es: [] for es in range(num_es)}
    for s in np.linspace(normalized_time[0], normalized_time[1]):
        eigval, eigvec = eigh(result[key].annealingH(s))
        seigval = (
            np.sort(eigval) * make_dimensionless + ising_params["c"] + qubo_constant
        )
        x.append(s)
        for es in y.keys():
            y[es].append(seigval[es])
    for es in y.keys():
        ax.errorbar(x=x, y=y[es])
    gap[key] = {es: np.array(y[es]) - np.array(y[0]) for es in y.keys()}
plt.draw()
plt.show()
print("ground state energy:", y[0][-1])
print("1st ex state energy:", y[1][-1])
eigval, eigvec = eigh(result[key].annealingH(s))
print("ground state eigvec:", np.round(eigvec[0]))
print("1st ex state eigvec:", np.round(eigvec[1]))

In [ ]:
fig = plt.figure()
ax = plt.axes()
for key in gap.keys():
    for es in gap[key].keys():
        ax.errorbar(x=x, y=gap[key][es], label=es)
# ax.legend()
plt.draw()
plt.show()
# print(f"e1-e0 gap[offset_range={min(gap.keys())}][{sum(mask)}]:", gap[min(gap.keys())][sum(mask)][-1])
# print(f"e2-e0 gap[offset_range={min(gap.keys())}][{sum(mask)+1}]:", gap[min(gap.keys())][sum(mask)+1][-1])

In [ ]:
mingap = {key: min(gap[key][1]) for key in gap.keys()}
plt.figure()
ax = plt.axes()
x = list(mingap.keys())
y = list(mingap.values())
ax.errorbar(x=x, y=y, marker="o")
plt.draw()
plt.show()
mingap

In [ ]:
sol.t

In [ ]:
sol_t = np.array(
    [
        -2.00000000e-01,
        -1.97359457e-01,
        -1.94717081e-01,
        -1.92074706e-01,
        -1.89432331e-01,
        -1.86789956e-01,
        -1.84147581e-01,
        -1.81505206e-01,
        -1.78862830e-01,
        -1.76220455e-01,
        -1.73578080e-01,
        -1.70935705e-01,
        -1.68293330e-01,
        -1.65650955e-01,
        -1.63008579e-01,
        -1.60366204e-01,
        -1.57723829e-01,
        -1.55081454e-01,
        -1.52439079e-01,
        -1.49796704e-01,
        -1.47154328e-01,
        -1.44511953e-01,
        -1.41869578e-01,
        -1.39227203e-01,
        -1.36584828e-01,
        -1.33942453e-01,
        -1.31300077e-01,
        -1.28657702e-01,
        -1.26015327e-01,
        -1.23372952e-01,
        -1.20730577e-01,
        -1.18088201e-01,
        -1.15445826e-01,
        -1.12803451e-01,
        -1.10161076e-01,
        -1.07518701e-01,
        -1.04876326e-01,
        -1.02233950e-01,
        -9.95915753e-02,
        -9.69492001e-02,
        -9.43068249e-02,
        -9.16644497e-02,
        -8.90220746e-02,
        -8.63796994e-02,
        -8.37373242e-02,
        -8.10949490e-02,
        -7.84525739e-02,
        -7.58101987e-02,
        -7.31678235e-02,
        -7.05254484e-02,
        -6.78830732e-02,
        -6.52406980e-02,
        -6.25983228e-02,
        -5.99559477e-02,
        -5.73135725e-02,
        -5.46711973e-02,
        -5.20288221e-02,
        -4.93864470e-02,
        -4.67440718e-02,
        -4.41016966e-02,
        -4.14593214e-02,
        -3.88169463e-02,
        -3.61745711e-02,
        -3.35321959e-02,
        -3.08898208e-02,
        -2.82474456e-02,
        -2.56050704e-02,
        -2.29626952e-02,
        -2.03203201e-02,
        -1.76779449e-02,
        -1.50355697e-02,
        -1.23783266e-02,
        -9.71735032e-03,
        -7.04928988e-03,
        -4.36874336e-03,
        -1.67456479e-03,
        1.04250049e-03,
        3.77006102e-03,
        6.50127812e-03,
        9.25566339e-03,
        1.20283285e-02,
        1.48273534e-02,
        1.76501374e-02,
        2.05140054e-02,
        2.33778734e-02,
        2.62417414e-02,
        2.91304796e-02,
        3.20440154e-02,
        3.50119659e-02,
        3.80480135e-02,
        4.10840611e-02,
        4.41477505e-02,
        4.72252581e-02,
        5.03636043e-02,
        5.37180389e-02,
        5.70724735e-02,
        6.04269081e-02,
        6.37813427e-02,
        6.71357773e-02,
        7.04902119e-02,
        7.39344394e-02,
        7.73786668e-02,
        8.08228942e-02,
        8.42671216e-02,
        8.77654683e-02,
        9.14456082e-02,
        9.51257481e-02,
        9.88058880e-02,
        1.02486028e-01,
        1.06183465e-01,
        1.09891681e-01,
        1.13660608e-01,
        1.17456918e-01,
        1.21275114e-01,
        1.25093310e-01,
        1.28985476e-01,
        1.32913223e-01,
        1.37086754e-01,
        1.41260285e-01,
        1.45433816e-01,
        1.49808289e-01,
        1.54182761e-01,
        1.58557234e-01,
        1.62931706e-01,
        1.67306179e-01,
        1.71680651e-01,
        1.76055124e-01,
        1.80434329e-01,
        1.84837684e-01,
        1.89281504e-01,
        1.93725324e-01,
        1.98270707e-01,
        2.02835705e-01,
        2.07421784e-01,
        2.12060540e-01,
        2.16754031e-01,
        2.21480960e-01,
        2.26260435e-01,
        2.31138269e-01,
        2.36016102e-01,
        2.40935621e-01,
        2.45855139e-01,
        2.50878617e-01,
        2.55942812e-01,
        2.61043847e-01,
        2.66144883e-01,
        2.71354519e-01,
        2.76614686e-01,
        2.81908533e-01,
        2.87264621e-01,
        2.92686092e-01,
        2.98150954e-01,
        3.03714662e-01,
        3.09290503e-01,
        3.14914642e-01,
        3.20586647e-01,
        3.26348107e-01,
        3.32141059e-01,
        3.37985539e-01,
        3.43882165e-01,
        3.49838277e-01,
        3.55858658e-01,
        3.61954524e-01,
        3.68071973e-01,
        3.74247846e-01,
        3.80480433e-01,
        3.86730949e-01,
        3.93107799e-01,
        3.99490627e-01,
        4.05959607e-01,
        4.12447516e-01,
        4.19004562e-01,
        4.25669957e-01,
        4.32335351e-01,
        4.39000746e-01,
        4.45813147e-01,
        4.52625548e-01,
        4.59437949e-01,
        4.66250350e-01,
        4.73062751e-01,
        4.79875152e-01,
        4.86687553e-01,
        4.93499954e-01,
        5.00312355e-01,
        5.07124756e-01,
        5.13937157e-01,
        5.20749558e-01,
        5.27561959e-01,
        5.34374360e-01,
        5.41186762e-01,
        5.47999163e-01,
        5.54811564e-01,
        5.60931823e-01,
        5.67060434e-01,
        5.73189045e-01,
        5.79317656e-01,
        5.85446267e-01,
        5.91574878e-01,
        5.97703489e-01,
        6.03176290e-01,
        6.08731648e-01,
        6.14287007e-01,
        6.19842365e-01,
        6.25397724e-01,
        6.30953082e-01,
        6.35915575e-01,
        6.40878068e-01,
        6.45840561e-01,
        6.50803053e-01,
        6.55765546e-01,
        6.60728039e-01,
        6.65690532e-01,
        6.70111427e-01,
        6.74532322e-01,
        6.78953217e-01,
        6.83374113e-01,
        6.87795008e-01,
        6.92215903e-01,
        6.96636798e-01,
        7.00566769e-01,
        7.04543736e-01,
        7.08520702e-01,
        7.12497668e-01,
        7.16474634e-01,
        7.20451601e-01,
        7.24428567e-01,
        7.28405533e-01,
        7.31947189e-01,
        7.35492325e-01,
        7.39037462e-01,
        7.42582599e-01,
        7.46127735e-01,
        7.49672872e-01,
        7.53218009e-01,
        7.56763146e-01,
        7.60308282e-01,
        7.63853419e-01,
        7.67022508e-01,
        7.70195436e-01,
        7.73368364e-01,
        7.76541292e-01,
        7.79714220e-01,
        7.82887148e-01,
        7.86060076e-01,
        7.89233003e-01,
        7.92405931e-01,
        7.95578859e-01,
        7.98751787e-01,
        8.01924715e-01,
        8.04766622e-01,
        8.07608529e-01,
        8.10450436e-01,
        8.13292343e-01,
        8.16134250e-01,
        8.18976157e-01,
        8.21818064e-01,
        8.24659971e-01,
        8.27501878e-01,
        8.30343785e-01,
        8.33185692e-01,
        8.36027599e-01,
        8.38869506e-01,
        8.41711413e-01,
        8.44267302e-01,
        8.46824993e-01,
        8.49382684e-01,
        8.51940375e-01,
        8.54498066e-01,
        8.57055757e-01,
        8.59613448e-01,
        8.62171139e-01,
        8.64728830e-01,
        8.67286521e-01,
        8.69844212e-01,
        8.72401903e-01,
        8.74959594e-01,
        8.77517285e-01,
        8.80074976e-01,
        8.82632667e-01,
        8.85190358e-01,
        8.87489968e-01,
        8.89789579e-01,
        8.92089190e-01,
        8.94388801e-01,
        8.96688412e-01,
        8.98988023e-01,
        9.01287634e-01,
        9.03587244e-01,
        9.05886855e-01,
        9.08186466e-01,
        9.10486077e-01,
        9.12785688e-01,
        9.15085299e-01,
        9.17384910e-01,
        9.19684520e-01,
        9.21984131e-01,
        9.24283742e-01,
        9.26583353e-01,
        9.28882964e-01,
        9.31182575e-01,
        9.33251229e-01,
        9.35320363e-01,
        9.37389497e-01,
        9.39458631e-01,
        9.41527765e-01,
        9.43596899e-01,
        9.45666033e-01,
        9.47735167e-01,
        9.49804301e-01,
        9.51873435e-01,
        9.53942569e-01,
        9.56011704e-01,
        9.58080838e-01,
        9.60149972e-01,
        9.62219106e-01,
        9.64288240e-01,
        9.66357374e-01,
        9.68426508e-01,
        9.70495642e-01,
        9.72564776e-01,
        9.74633910e-01,
        9.76703044e-01,
        9.78772178e-01,
        9.80841312e-01,
        9.82910446e-01,
        9.84979580e-01,
        9.87048714e-01,
        9.88909891e-01,
        9.90771596e-01,
        9.92633301e-01,
        9.94495006e-01,
        9.96356710e-01,
        9.98218415e-01,
        1.00008012e00,
        1.00194183e00,
        1.00380353e00,
        1.00566523e00,
        1.00752694e00,
        1.00938864e00,
        1.01125035e00,
        1.01311205e00,
        1.01497376e00,
        1.01683546e00,
        1.01869717e00,
        1.02055887e00,
        1.02242058e00,
        1.02428228e00,
        1.02614399e00,
        1.02800569e00,
        1.02986740e00,
        1.03172910e00,
        1.03359081e00,
        1.03545251e00,
        1.03731422e00,
        1.03917592e00,
        1.04103763e00,
        1.04289933e00,
        1.04476104e00,
        1.04662274e00,
        1.04848445e00,
        1.05034615e00,
        1.05220786e00,
        1.05406956e00,
        1.05593127e00,
        1.05779297e00,
        1.05965468e00,
        1.06151638e00,
        1.06337809e00,
        1.06523979e00,
        1.06710149e00,
        1.06896320e00,
        1.07082490e00,
        1.07268661e00,
        1.07454831e00,
        1.07641002e00,
        1.07827172e00,
        1.08013343e00,
        1.08199513e00,
        1.08385684e00,
        1.08571854e00,
        1.08758025e00,
        1.08944195e00,
        1.09130366e00,
        1.09316536e00,
        1.09502707e00,
        1.09688877e00,
        1.09875048e00,
        1.10061218e00,
        1.10247389e00,
        1.10433559e00,
        1.10619730e00,
        1.10805900e00,
        1.10992071e00,
        1.11178241e00,
        1.11364412e00,
        1.11550582e00,
        1.11736753e00,
        1.11922923e00,
        1.12109094e00,
        1.12295264e00,
        1.12481435e00,
        1.12667605e00,
        1.12853776e00,
        1.13039946e00,
        1.13226116e00,
        1.13412287e00,
        1.13598457e00,
        1.13784628e00,
        1.13970798e00,
        1.14156969e00,
        1.14343139e00,
        1.14529310e00,
        1.14715480e00,
        1.14901651e00,
        1.15087821e00,
        1.15273992e00,
        1.15460162e00,
        1.15646333e00,
        1.15832503e00,
        1.16018674e00,
        1.16204844e00,
        1.16391015e00,
        1.16577185e00,
        1.16763356e00,
        1.16949526e00,
        1.17135697e00,
        1.17321867e00,
        1.17508038e00,
        1.17694208e00,
        1.17880379e00,
        1.18066549e00,
        1.18252720e00,
        1.18438890e00,
        1.18625061e00,
        1.18811231e00,
        1.18997402e00,
        1.19183572e00,
        1.19369743e00,
        1.19555913e00,
        1.19742083e00,
        1.19928254e00,
        1.20000000e00,
    ]
)

In [ ]:
a = np.abs(sol.t - sol_t)
print(a.mean(), a.sum())